In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
import sklearn.preprocessing

def normalize_data(df):
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    
    
    for column in numeric_columns:  # Use 'df' instead of 'normalized_df' to iterate over its columns
        df[column] = min_max_scaler.fit_transform(df[column].values.reshape(-1, 1))  # Correct access to columns
    return df


In [13]:
project_dir = "/home/jupyter-tfg2425paula"
os.chdir(project_dir)
data_dir = os.path.join(project_dir, "raw_data")
input_processed_data_dir = os.path.join(project_dir, 'raw_data/options_and_combinations')

In [34]:
from openpyxl import load_workbook

stocks = 'AAPL_MSFT_AMZN_NVDA_SPX'
filename = f'{stocks}_options.xlsx'
workbook = load_workbook(os.path.join(input_processed_data_dir, filename))
sheet = workbook.active
data = []
for row in sheet.iter_rows(values_only=True):  # Iterate through rows, extract values
    data.append(row)

# Convert to pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

df.head()

,Date,AAPL_Close,AAPL_CALL_OM,AAPL_CALL_O1,AAPL_CALL_OY,AAPL_CALL_OI,AAPL_CALL_VM,AAPL_PUT_OM,AAPL_PUT_O1,AAPL_PUT_OY,...,SPX_CALL_OM,SPX_CALL_O1,SPX_CALL_OY,SPX_CALL_OI,SPX_CALL_VM,SPX_PUT_OM,SPX_PUT_O1,SPX_PUT_OY,SPX_PUT_OI,SPX_PUT_VM
0,2009-06-18,135.88,6.80,0.4063,0.4254,920448,31936.0,5.70,0.3943,0.4314,...,24.3,0.2568,0.2698,3816626,360148.0,28.16,0.2656,0.2939,5607238,468262.0
1,2009-06-19,139.48,5.40,0.3606,0.4244,687080,40040.0,5.96,0.3654,0.4145,...,23.0,0.2335,0.2635,3889631,211146.0,25.00,0.2524,0.2950,5775618,296620.0
2,2009-06-22,137.37,6.90,0.3852,0.4156,708547,25673.0,4.45,0.3845,0.4333,...,24.4,0.2880,0.2804,3806131,195176.0,26.90,0.2799,0.3056,5685968,312092.0
3,2009-06-23,134.01,4.52,0.3653,0.4180,725825,24184.0,5.60,0.3747,0.4259,...,24.0,0.2734,0.2752,3755289,117641.0,25.00,0.2757,0.3059,5287312,312995.0
4,2009-06-24,136.22,5.45,0.3513,0.4088,732078,16474.0,4.10,0.3464,0.4138,...,24.0,0.2707,0.2763,4028277,225367.0,22.60,0.2595,0.2936,6065984,255662.0


In [35]:
security = 'AAPL'

df[f'RETURN_{security}'] = (df[f'{security}_Close'] - df[f'{security}_Close'].shift(1)) / df[f'{security}_Close'].shift(1)
df[f'TARGET_{security}'] = (df[f'{security}_Close'].shift(-1) - df[f'{security}_Close'] > 0).astype(float)

df = df.drop(columns = ['Date'])
    
df = df.fillna(0)
df = df.rename(columns={f'TARGET_{security}': 'Target'})

normalized_df = normalize_data(df)

In [36]:
target_df = normalized_df['Target']
features_df = normalized_df.drop(columns = ["Target"])
target_df

0       1.0
1       0.0
2       0.0
3       1.0
4       1.0
       ... 
4023    1.0
4024    0.0
4025    1.0
4026    1.0
4027    0.0
Name: Target, Length: 4028, dtype: float64

In [38]:
def replace_with_average(df, target_value='ç'):
    for col in df.columns:
        for i in range(len(df[col])):
            if df[col].iloc[i] == target_value:
                prev_value = pd.to_numeric(df[col].iloc[i - 1], errors='coerce') if i > 0 else np.nan
                next_value = pd.to_numeric(df[col].iloc[i + 1], errors='coerce') if i < len(df[col]) - 1 else np.nan
                
                # Calculate the average, ignoring NaNs
                avg_value = np.nanmean([prev_value, next_value])
                df[col].iloc[i] = avg_value

replace_with_average(features_df)
features_df = features_df.apply(pd.to_numeric, errors='coerce')

/tmp/ipykernel_18610/3043059538.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[col].iloc[i] = avg_value
/tmp/ipykernel_18610/3043059538.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [39]:
features_df

,AAPL_Close,AAPL_CALL_OM,AAPL_CALL_O1,AAPL_CALL_OY,AAPL_CALL_OI,AAPL_CALL_VM,AAPL_PUT_OM,AAPL_PUT_O1,AAPL_PUT_OY,AAPL_PUT_OI,...,SPX_CALL_O1,SPX_CALL_OY,SPX_CALL_OI,SPX_CALL_VM,SPX_PUT_OM,SPX_PUT_O1,SPX_PUT_OY,SPX_PUT_OI,SPX_PUT_VM,RETURN_AAPL
0,0.074532,0.166161,0.083655,0.145124,0.109092,0.007441,0.136778,0.122097,0.052680,0.130542,...,0.249836,0.448234,0.495566,0.381835,0.095962,0.282048,0.312108,0.471325,0.202485,0.877078
1,0.080416,0.123711,0.070093,0.144516,0.081433,0.009329,0.144681,0.109138,0.048656,0.094416,...,0.219198,0.429218,0.505046,0.223861,0.085193,0.263833,0.314138,0.485479,0.128264,0.904260
2,0.076967,0.169193,0.077393,0.139166,0.083977,0.005982,0.098784,0.117702,0.053133,0.098452,...,0.290861,0.480229,0.494204,0.206929,0.091668,0.301780,0.333702,0.477943,0.134954,0.861557
3,0.071475,0.097029,0.071488,0.140625,0.086025,0.005635,0.133739,0.113308,0.051371,0.106272,...,0.271663,0.464534,0.487602,0.124725,0.085193,0.295985,0.334256,0.444433,0.135345,0.851982
4,0.075087,0.125227,0.067333,0.135032,0.086766,0.003838,0.088146,0.100619,0.048489,0.107550,...,0.268113,0.467854,0.523048,0.238938,0.077015,0.273630,0.311554,0.509886,0.110553,0.893998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,0.225557,0.146452,0.024304,0.036661,0.338658,0.017701,0.095745,0.030087,0.004835,0.389089,...,0.085733,0.131301,0.770164,0.314144,0.264440,0.094108,0.062938,0.789389,0.174400,0.878247
4024,0.226733,0.117647,0.022405,0.039701,0.339928,0.017178,0.120669,0.031163,0.005382,0.392154,...,0.091124,0.134319,0.776531,0.198164,0.271937,0.101145,0.065338,0.798096,0.150884,0.880314
4025,0.225949,0.151001,0.023176,0.039822,0.340315,0.018082,0.091793,0.032329,0.005192,0.395588,...,0.084024,0.130697,0.783500,0.298040,0.268018,0.095902,0.064230,0.803673,0.175772,0.874927
4026,0.228155,0.115221,0.018755,0.032892,0.342445,0.017040,0.095745,0.035916,0.004454,0.397869,...,0.063774,0.121340,0.785831,0.151766,0.235134,0.076721,0.058878,0.811522,0.106681,0.883139


In [40]:
pca = PCA(n_components=0.90) # set 95% total var
pca_data = pca.fit_transform(features_df)
num_components = pca.n_components_
explained_variance = pca.explained_variance_ratio_

print(f'Número de componentes seleccionados: {num_components}')
print(f'Varianza explicada por cada componente: {explained_variance}')

Número de componentes seleccionados: 1
Varianza explicada por cada componente: [0.95026344]


In [41]:
# Obtener los nombres de las columnas originales (del DataFrame antes de PCA)
original_features = features_df.columns

# Crear un DataFrame para las cargas de los componentes principales
pca_components = pd.DataFrame(pca.components_, columns=original_features, index=[f'PC{i+1}' for i in range(num_components)])
# pca_components.to_csv(os.path.join(pca_folder, 'pca_components.csv'))
print(pca_components)


     AAPL_Close  AAPL_CALL_OM  AAPL_CALL_O1  AAPL_CALL_OY  AAPL_CALL_OI  \
PC1   -0.019765     -0.008436      0.002261      0.000506      0.016429   

     AAPL_CALL_VM  AAPL_PUT_OM  AAPL_PUT_O1  AAPL_PUT_OY  AAPL_PUT_OI  ...  \
PC1      0.002285    -0.009806     0.003007    -0.000064     0.024039  ...   

     SPX_CALL_O1  SPX_CALL_OY  SPX_CALL_OI  SPX_CALL_VM  SPX_PUT_OM  \
PC1     0.005718     0.003414    -0.005204    -0.008262    0.023414   

     SPX_PUT_O1  SPX_PUT_OY  SPX_PUT_OI  SPX_PUT_VM  RETURN_AAPL  
PC1    0.006297     0.00431   -0.003618   -0.004325    -0.000142  

[1 rows x 56 columns]


In [44]:
import pandas as pd
from factor_analyzer import Rotator
import numpy as np

# Assuming you already have:
# `pca` - the fitted PCA object
# `pca_components` - the DataFrame containing PCA loadings matrix
# `pca_data` - the transformed PCA scores from your data

# Extract loadings matrix as a NumPy array
pca_loadings = pca_components.values

# Initialize the rotator for oblimin rotation
rotator = Rotator(method='oblimin')

# Apply the oblimin rotation on the loadings matrix
rotated_loadings = rotator.fit_transform(pca_loadings)

# Convert the rotated loadings to a DataFrame
rotated_pca_components = pd.DataFrame(
    rotated_loadings,
    columns=pca_components.columns,
    index=pca_components.index
)

# Now, transform the original PCA data using the rotated loadings
# This is done via matrix multiplication between the PCA-transformed data and the rotated loadings
rotated_pca_data = np.dot(features_df, rotated_loadings.T)

# Convert the transformed data to a DataFrame
rotated_pca_features_df = pd.DataFrame(
    rotated_pca_data,
    columns=[f'Rotated_PC{i+1}' for i in range(rotated_pca_data.shape[1])]
)

print("Rotated Loadings Matrix:")
rotated_pca_components = pd.DataFrame(rotated_loadings, columns=pca_components.columns, index=pca_components.index)
print(rotated_pca_components)

print("\nRotated PCA Data:")
print(rotated_pca_features_df)

Rotated Loadings Matrix:
       AAPL_Close  AAPL_CALL_OM  AAPL_CALL_O1  AAPL_CALL_OY  AAPL_CALL_OI  \
PC1 -3.542037e-08 -1.610894e-08  4.374551e-09  9.797994e-10  3.015326e-08   

     AAPL_CALL_VM   AAPL_PUT_OM   AAPL_PUT_O1   AAPL_PUT_OY   AAPL_PUT_OI  \
PC1  4.422118e-09 -1.863278e-08  5.815111e-09 -1.231680e-10  4.153371e-08   

     ...   SPX_CALL_O1   SPX_CALL_OY   SPX_CALL_OI   SPX_CALL_VM  \
PC1  ...  1.100351e-08  6.598231e-09 -1.002539e-08 -1.578559e-08   

       SPX_PUT_OM    SPX_PUT_O1    SPX_PUT_OY    SPX_PUT_OI    SPX_PUT_VM  \
PC1  4.068776e-08  1.210184e-08  8.317131e-09 -6.989873e-09 -8.346435e-09   

      RETURN_AAPL  
PC1 -2.741566e-10  

[1 rows x 56 columns]

Rotated PCA Data:
      Rotated_PC1
0            0.77
1            0.77
2            0.65
3            0.66
4            0.52
...           ...
4023         9.00
4024         9.00
4025         8.38
4026         8.10
4027         7.55

[4028 rows x 1 columns]


In [46]:
rotated_final_df = pd.concat([rotated_pca_features_df, target_df.reset_index(drop=True)], axis=1)
data_dir = os.path.join(project_dir, "raw_data")
output_filename = f'rotated_{stocks}_options.csv'
rotated_final_df.to_csv(os.path.join(data_dir, output_filename), index=False)